In [81]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
#from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Model, load_model, save_model
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [83]:
data = pd.read_csv('gpt2_baseline_aidu.csv',encoding = 'utf-8')
data

,type,sess_id,speaker_1_id,query,speaker_2_id,reply,system_response,gen_reply
0,test,4-06496-4,22032,한달이 흘렀네요. 저는 다시 대구로 복귀했답니다. 잘 지내셨나요?,22658,"일 때문에 제법 바빴지만, 그래도 잘 지냈어요. 주말마다 공원에도 가고. 안 그래도...",네 잘 지냈어요. 저는 요즘도 가끔 느긋해지네요.,네 잘 지냈어요. 저는 요즘도 가끔 느긋해지네요.
1,test,4-45996-4,11181,안녕하세요! 5주만이네요 그동안 잘 지내셨나요?,11700,반가워요! 그동안 잘 지냈어요.,네 잘 지냈어요! 저는 요즘에 코로나에 걸려서 격리 중이에요,네 잘 지냈어요! 저는 요즘에 코로나에 걸려서 격리 중이에요
2,test,4-45996-4,11181,저는 최근에 영화를 보고 왔어요! 근데 주말이었는데도 사람이 적더라구요,11700,굉장한데요? 주말에 사람이 없었다니 무슨 일이래요.,아하 그렇군요! 저는 요즘에 힙합 음악을 자주 듣고 있어요!,아하 그렇군요! 저는 요즘에 힙합 음악을 자주 듣고 있어요!
3,test,4-45996-4,11181,아무래도 인기있는 상업영화가 상영하고 있지 않아서 그런거 같아요!,11700,저도 요즘 영화가 안끌리더라고요. 요즘은 굳이 영화관에 가지 않아도 볼 수 있는 매...,그렇군요! 저는 이만 일이 있어서 가볼게요!,그렇군요! 저는 이만 일이 있어서 가볼게요!
4,test,4-45996-4,11181,맞아요! 최근에 @브랜드@에서 에놀라홈즈라는 영화를 봤는데 재밌더라구요,11700,오오. 한 번도 들어보지 못한 영화라 흥미가 가는데요?,저도 <unk>브랜드<unk>영화 봤어요!!!!!!!!!!!!!!!!!!!!!!!!...,저도 브랜드영화 봤어요!!!
...,...,...,...,...,...,...,...,...
2123,test,2-43090-2,50232,저는 일본이 비자가 풀려서 조만간 일본을 가려고 합니다.,32397,요즘 일본가는 분들이 많은것 같아요.. 저는 휴양차 아이들과 괌에 갈까해서 알아보고...,일본 좋죠. 저도 일본 여행 가고 싶네요.,일본 좋죠. 저도 일본 여행 가고 싶네요.
2124,test,2-43090-2,50232,괌도 너무 좋죠! 저는 일본에 지인이 있어서 가보려고 하는데 온천 패키지로 갈까 합니다.,32397,지인도 있고 온천 너무 좋지요.. 요즘 엔화도 떨어져서 일본가지 너무 좋을것 같습니...,온천 너무 좋죠. 저도 일본 여행 가고 싶네요.,온천 너무 좋죠. 저도 일본 여행 가고 싶네요.
2125,test,2-43090-2,50232,달러가 올라서 면세점은 패스해야 겠지만 여행 생각만으로도 즐겁네요,32397,저도 여행상품 오늘까지만 보고 결제 해야할까봅니다 빨리 여행가고 싶네요,"네네, 저도 여행 계획 짜러 가야겠어요. 다음에 또 이야기 나눠요.","네네, 저도 여행 계획 짜러 가야겠어요. 다음에 또 이야기 나눠요."
2126,test,2-43090-2,50232,이제 노선도 많이 늘어나고 하면 업체들끼리 경쟁붙어 조금더 싸지길 기대해 봅니다.,32397,네 저는 막상 여행 가서도 좋지만 비행기 타러 갈때가 가장 들뜨는것 같아요.,네네 그래야겠어요~ 저는 이만 일이 있어서 가볼게요~,네네 그래야겠어요~ 저는 이만 일이 있어서 가볼게요~


In [84]:
data['system_response'] = data['system_response'].str.replace('<unk>', '@')

In [85]:
import sys
import re

special = re.compile(r'[^\sA-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ@?!~,.ᄒ><\^+]')
doublespace_pattern = re.compile('\s+')
repeatchars_pattern = re.compile('(\D)\\1{2,}')

def repeat_normalize(sent, num_repeats=2):
    sent = special.sub('',sent)
    if num_repeats > 0:
        sent = repeatchars_pattern.sub('\\1' * num_repeats, sent)
    sent = doublespace_pattern.sub(' ', sent)
    return sent.strip()
def proc_reply(reply):
    proc_text = re.sub('(<pad>|<unk>|<u>)', '', reply)
    proc_text = repeat_normalize(proc_text, num_repeats=3)
    return proc_text

In [88]:
data['gen_reply'] = data['system_response'].apply(proc_reply)
data

,type,sess_id,speaker_1_id,query,speaker_2_id,reply,system_response,gen_reply
0,test,4-06496-4,22032,한달이 흘렀네요. 저는 다시 대구로 복귀했답니다. 잘 지내셨나요?,22658,"일 때문에 제법 바빴지만, 그래도 잘 지냈어요. 주말마다 공원에도 가고. 안 그래도...",네 잘 지냈어요. 저는 요즘도 가끔 느긋해지네요.,네 잘 지냈어요. 저는 요즘도 가끔 느긋해지네요.
1,test,4-45996-4,11181,안녕하세요! 5주만이네요 그동안 잘 지내셨나요?,11700,반가워요! 그동안 잘 지냈어요.,네 잘 지냈어요! 저는 요즘에 코로나에 걸려서 격리 중이에요,네 잘 지냈어요! 저는 요즘에 코로나에 걸려서 격리 중이에요
2,test,4-45996-4,11181,저는 최근에 영화를 보고 왔어요! 근데 주말이었는데도 사람이 적더라구요,11700,굉장한데요? 주말에 사람이 없었다니 무슨 일이래요.,아하 그렇군요! 저는 요즘에 힙합 음악을 자주 듣고 있어요!,아하 그렇군요! 저는 요즘에 힙합 음악을 자주 듣고 있어요!
3,test,4-45996-4,11181,아무래도 인기있는 상업영화가 상영하고 있지 않아서 그런거 같아요!,11700,저도 요즘 영화가 안끌리더라고요. 요즘은 굳이 영화관에 가지 않아도 볼 수 있는 매...,그렇군요! 저는 이만 일이 있어서 가볼게요!,그렇군요! 저는 이만 일이 있어서 가볼게요!
4,test,4-45996-4,11181,맞아요! 최근에 @브랜드@에서 에놀라홈즈라는 영화를 봤는데 재밌더라구요,11700,오오. 한 번도 들어보지 못한 영화라 흥미가 가는데요?,저도 @브랜드@영화 봤어요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,저도 @브랜드@영화 봤어요!!!
...,...,...,...,...,...,...,...,...
2123,test,2-43090-2,50232,저는 일본이 비자가 풀려서 조만간 일본을 가려고 합니다.,32397,요즘 일본가는 분들이 많은것 같아요.. 저는 휴양차 아이들과 괌에 갈까해서 알아보고...,일본 좋죠. 저도 일본 여행 가고 싶네요.,일본 좋죠. 저도 일본 여행 가고 싶네요.
2124,test,2-43090-2,50232,괌도 너무 좋죠! 저는 일본에 지인이 있어서 가보려고 하는데 온천 패키지로 갈까 합니다.,32397,지인도 있고 온천 너무 좋지요.. 요즘 엔화도 떨어져서 일본가지 너무 좋을것 같습니...,온천 너무 좋죠. 저도 일본 여행 가고 싶네요.,온천 너무 좋죠. 저도 일본 여행 가고 싶네요.
2125,test,2-43090-2,50232,달러가 올라서 면세점은 패스해야 겠지만 여행 생각만으로도 즐겁네요,32397,저도 여행상품 오늘까지만 보고 결제 해야할까봅니다 빨리 여행가고 싶네요,"네네, 저도 여행 계획 짜러 가야겠어요. 다음에 또 이야기 나눠요.","네네, 저도 여행 계획 짜러 가야겠어요. 다음에 또 이야기 나눠요."
2126,test,2-43090-2,50232,이제 노선도 많이 늘어나고 하면 업체들끼리 경쟁붙어 조금더 싸지길 기대해 봅니다.,32397,네 저는 막상 여행 가서도 좋지만 비행기 타러 갈때가 가장 들뜨는것 같아요.,네네 그래야겠어요~ 저는 이만 일이 있어서 가볼게요~,네네 그래야겠어요~ 저는 이만 일이 있어서 가볼게요~


In [89]:
data.to_csv('test2.csv')